In [1]:
from keras.utils import np_utils
import numpy as np
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

# Display progress logs on stdout
#import logging
#logging.basicConfig(level=logging.DEBUG,
#                    format='%(asctime)s %(levelname)s %(message)s')

from sklearn.datasets import load_files
import pandas as pd
pd.set_option("display.max_colwidth", 75)

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
basedir="/home/tutysara/src/myprojects/dog-project/dogImages"
train_idx_path = basedir+ "/train_list.txt"
valid_idx_path = basedir+ "/valid_list.txt"
test_idx_path = basedir+ "/test_list.txt"


col_names = ["X", "y"]
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
train_data_df.y = train_data_df.y-1
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
valid_data_df.y = valid_data_df.y-1
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)
test_data_df.y = test_data_df.y-1

In [3]:
test_data_df.head()

,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [4]:
print(train_data_df.shape)
print(valid_data_df.shape)
print(test_data_df.shape)

print(train_data_df.shape[0] + valid_data_df.shape[0] + test_data_df.shape[0])
sample_df = test_data_df[:5]
sample_df

(6680, 2)
(835, 2)
(836, 2)
8351


,X,y
0,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05148.jpg,74
1,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05143.jpg,74
2,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05164.jpg,74
3,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05129.jpg,74
4,test/075.Glen_of_imaal_terrier/Glen_of_imaal_terrier_05149.jpg,74


In [5]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [6]:
#definition of data generator
def df_data_generator(df, batch_size=32, num_classes=3, shuffle=False, basedir=".", transformer=None):    
    while True:
        if shuffle:
            df = df.sample(frac=1)
            
        X_file_name = df.X.apply(lambda x: basedir+"/"+x) 
        y = np_utils.to_categorical(df.y, num_classes)
    # infinitely serve batches
        max_range = (1+ df.shape[0]//batch_size)
        for i in range(max_range):
            if transformer:
                yield transformer(X_file_name[i*batch_size : (i+1)*batch_size]).astype('float32'), y[i*batch_size : (i+1)*batch_size]
            else:
                yield X_file_name[i*batch_size : (i+1)*batch_size], y[i*batch_size : (i+1)*batch_size]
                

from keras.applications.mobilenet import MobileNet
import keras.applications.mobilenet as mobilenet
def transformer(x):
    return mobilenet.preprocess_input(paths_to_tensor(x))


In [7]:
bnf_train_name = 'bottleneck_features_mobilenet_train.bclz'
bnf_valid_name = 'bottleneck_features_mobilenet_valid.bclz'
bnf_test_name = 'bottleneck_features_mobilenet_test.bclz' 

In [24]:
import bcolz
def bcolz_prediction_writer(gen, steps, model, dirname ):
    X = None; y = None
    for i in range(steps):
        X_batch, y_batch = next(gen)
        X_out = model.predict(X_batch)
        if i== 0:
            X = bcolz.carray(X_out, rootdir=dirname, mode='w')
            y = y_batch
        else:
            X.append(X_out)
            y = np.concatenate((y,y_batch))
    X .flush()
    return X, y

In [28]:
from keras.applications.mobilenet import MobileNet
import time

# load train, test, and validation datasets
batch_size=32
train_gen = df_data_generator(train_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
valid_gen = df_data_generator(valid_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)
test_gen = df_data_generator(test_data_df, batch_size=batch_size, transformer=transformer, basedir=basedir, num_classes=133)



mobilenet_feature_ext = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
mobilenet_feature_ext._make_predict_function()

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_valid.bclz'
bnf_valid_data,  bnf_valid_labels = bcolz_prediction_writer(gen=valid_gen, 
                        steps=(1+(valid_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_valid_name)
print("Took {} seconds to calculate bnf_valid_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_test.bclz'
bnf_test_data,  bnf_test_labels = bcolz_prediction_writer(gen=test_gen, 
                        steps=(1+(test_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_test_name)
print("Took {} seconds to calculate bnf_test_data".format(time.time()-s))

s= time.time()
!rm -rf 'bottleneck_features_mobilenet_train.bclz'
bnf_train_data,  bnf_train_labels = bcolz_prediction_writer(gen=train_gen, 
                        steps=(1+(train_data_df.shape[0]//batch_size)),
                        model=mobilenet_feature_ext,
                        dirname=bnf_train_name)
print("Took {} seconds to calculate bottleneck_features_mobilenet_train".format(time.time()-s))


Took 12.09251093864441 seconds to calculate bnf_valid_data
Took 11.467346668243408 seconds to calculate bnf_test_data
Took 97.50013709068298 seconds to calculate bottleneck_features_mobilenet_train


In [30]:
print(bnf_valid_data.shape)
print(bnf_test_data.shape)
print(bnf_train_data.shape)


(835, 7, 7, 1024)
(836, 7, 7, 1024)
(6680, 7, 7, 1024)


In [31]:
print(bnf_valid_labels.shape)
print(bnf_test_labels.shape)
print(bnf_train_labels.shape)

(835, 133)
(836, 133)
(6680, 133)


In [32]:
def bnf_data_generator(bnf_data, bnf_labels, batch_size=32):
    while True:
        max_range = (1 + bnf_data.shape[0]//batch_size)
        for i in range(max_range):
            yield bnf_data[i*batch_size : (i+1)*batch_size], bnf_labels[i*batch_size : (i+1)*batch_size]                

In [35]:
## make a generator of loaded data
batch_size = 256
train_gen_fex =bnf_data_generator(bnf_train_data, bnf_train_labels, batch_size=batch_size)
valid_gen_fex =bnf_data_generator(bnf_valid_data, bnf_valid_labels, batch_size=batch_size)
test_gen_fex = bnf_data_generator(bnf_test_data, bnf_test_labels, batch_size=batch_size) 

In [34]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers

top_model = Sequential()
#model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 1024)))
top_model.add(Dropout(0.2))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.2))
top_model.add(Dense(133, activation='softmax'))

top_model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])

top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               68229     
Total params: 593,029
Trainable params: 593,029
Non-trainable params: 0
_________________________________________________________________


In [37]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
top_model.fit_generator(train_gen_fex,
          steps_per_epoch= (1 + (train_data_df.shape[0]// batch_size)),
          epochs=25,
          validation_data=valid_gen_fex,
          validation_steps= (1 + (valid_data_df.shape[0] // batch_size)),
          callbacks=[early_stopping, checkpointer])

Epoch 1/25
27/27 [==============================] - 4s 159ms/step - loss: 5.7526 - acc: 0.0000e+00 - val_loss: 4.8178 - val_acc: 0.0491
Epoch 2/25
27/27 [==============================] - 2s 87ms/step - loss: 4.7848 - acc: 0.0462 - val_loss: 4.5483 - val_acc: 0.1054
Epoch 3/25
27/27 [==============================] - 3s 101ms/step - loss: 4.5130 - acc: 0.0715 - val_loss: 3.9692 - val_acc: 0.2216
Epoch 4/25
27/27 [==============================] - 3s 95ms/step - loss: 3.9609 - acc: 0.1169 - val_loss: 3.2907 - val_acc: 0.2874
Epoch 5/25
27/27 [==============================] - 3s 103ms/step - loss: 3.4302 - acc: 0.1758 - val_loss: 2.7141 - val_acc: 0.3892
Epoch 6/25
27/27 [==============================] - 3s 98ms/step - loss: 2.9019 - acc: 0.2680 - val_loss: 2.1456 - val_acc: 0.5114
Epoch 7/25
27/27 [==============================] - 3s 101ms/step - loss: 2.4403 - acc: 0.3453 - val_loss: 1.7608 - val_acc: 0.6036
Epoch 8/25
27/27 [==============================] - 3s 97ms/step - loss: 2.

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
top_model.fit(train_data[:250], train_labels[:250],
          epochs=25,
          batch_size=64,
          validation_data=(valid_data[:150], valid_labels[:150]),
          callbacks=[early_stopping])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.topmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
top_model.fit_generator(train_gen_fex,
          steps_per_epoch= (train_data_df.shape[0]// batch_size),
          epochs=25,
          validation_data=valid_gen_fex,
          validation_steps= (valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

In [40]:
# fine tune on full model
from keras import optimizers
mobilenet_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# CREATE AN "REAL" MODEL FROM Mobilenet
# BY COPYING ALL THE LAYERS OF Mobilenet
model = Sequential()
for l in mobilenet_model.layers:
    model.add(l)


# CONCATENATE THE TWO MODELS
model.add(top_model)

# LOCK THE TOP CONV LAYERS
#for layer in model.layers:
#    layer.trainable = False

# COMPILE THE MODEL
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [41]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fullmodel.mobilenet.hdf5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
model.fit_generator(train_gen,
          steps_per_epoch= (train_data_df.shape[0]// batch_size),
          epochs=25,
          validation_data=valid_gen,
          validation_steps= (valid_data_df.shape[0] // batch_size),
          callbacks=[early_stopping, checkpointer])

Epoch 1/25
26/26 [==============================] - 24s 933ms/step - loss: 6.3531 - acc: 0.0637 - val_loss: 0.5555 - val_acc: 0.8229
Epoch 2/25
26/26 [==============================] - 9s 356ms/step - loss: 6.0963 - acc: 0.0793 - val_loss: 0.4923 - val_acc: 0.8438
Epoch 3/25
26/26 [==============================] - 9s 332ms/step - loss: 6.4498 - acc: 0.0577 - val_loss: 0.6867 - val_acc: 0.7604
Epoch 4/25
26/26 [==============================] - 8s 326ms/step - loss: 6.2235 - acc: 0.0649 - val_loss: 0.7042 - val_acc: 0.8229
Epoch 5/25
26/26 [==============================] - 9s 331ms/step - loss: 5.9292 - acc: 0.0829 - val_loss: 0.9112 - val_acc: 0.7396
Epoch 6/25
26/26 [==============================] - 9s 363ms/step - loss: 6.1385 - acc: 0.0613 - val_loss: 0.9291 - val_acc: 0.7188
Epoch 00006: early stopping
